# Apert与正常群体间的二分类模型

In [3]:
import os
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.callbacks import Callback
from tensorflow import keras
from sklearn.metrics import f1_score, recall_score, precision_score

数据集路径

In [4]:
TRAIN_SET_DIR = './dataset/train'
TEST_SET_DIR = './dataset/test'

构造generator，避免一次读取全部图片导致占满内存

In [5]:
image_processor = ImageDataGenerator(validation_split=0.1)
train_generator = image_processor.flow_from_directory(
    directory=TRAIN_SET_DIR,
    target_size=(112, 96),
    batch_size=16,
    subset='training'
)
validation_generator = image_processor.flow_from_directory(
    directory=TRAIN_SET_DIR,
    target_size=(112, 96),
    batch_size=sum([
      len(os.listdir(os.path.join(TEST_SET_DIR, class_name)))
      for class_name in os.listdir(TEST_SET_DIR)
    ]),
    subset='validation'
)
test_generator = image_processor.flow_from_directory(
    directory=TEST_SET_DIR,
    target_size=(112, 96),
    batch_size=16
)

Found 244 images belonging to 2 classes.
Found 26 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


加载经过训练的预训练MobileFaceNet模型

In [6]:
mobile_facenet = load_model('../../mobile_facenet.h5')

改造模型结构使其适应当前问题

In [7]:
top_layer = Dense(
    2,
    kernel_initializer='he_normal',
    activation='softmax'
)(mobile_facenet.output)

In [8]:
model = Model(inputs=mobile_facenet.input, outputs=top_layer)

增加checkpoint，保存中间结果

In [9]:
model_checkpoint_callback = ModelCheckpoint(
        'saved_models/top_layer_trained_weights.{epoch:02d}-{val_accuracy:.2f}.h5',
        monitor='val_accuracy',
        mode='max',
        verbose=1,
        save_best_only=True
    )

In [10]:
batches_per_epoch = train_generator.n // train_generator.batch_size

使用tensorboard

In [11]:
tensorboard_callback = TensorBoard(
        log_dir='logs',
        batch_size=16
    )

监控准确率、查准率、查全率、auc

In [12]:
METRICS = [
  # keras.metrics.TruePositives(name='tp'),
  # keras.metrics.FalsePositives(name='fp'),
  # keras.metrics.TrueNegatives(name='tn'),
  # keras.metrics.FalseNegatives(name='fn'), 
  keras.metrics.BinaryAccuracy(name='accuracy'),
  keras.metrics.Precision(name='precision'),
  keras.metrics.Recall(name='recall'),
  keras.metrics.AUC(name='auc'),
]

编译模型

In [13]:
model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics = METRICS
    )

对模型进行训练

In [14]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=batches_per_epoch,
    epochs=10,
    callbacks=[
            # mae_callback,
            # early_stopping_callback,
            # metrics,
            model_checkpoint_callback,
            tensorboard_callback
        ],
    validation_data=validation_generator
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
 1/15 [=>............................] - ETA: 0s - loss: 3.4252 - accuracy: 0.6875 - precision: 0.6875 - recall: 0.6875 - auc: 0.6211WARNING:tensorflow:From D:\program\Anaconda\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
15/15 [==============================] - ETA: 0s - loss: 1.0529 - accuracy: 0.9167 - precision: 0.9167 - recall: 0.9167 - auc: 0.9564
Epoch 00001: val_accuracy improved from -inf to 0.61538, saving model to saved_models\top_layer_trained_weights.01-0.62.h5
15/15 [==============================] - 18s 1s/step - loss: 1.0529 - accuracy: 0.9167 - precision: 0.9167 - recall: 0.9167 - auc: 0.9564 - val_loss: 1.8015 - val_accuracy: 0.6154 - val_precision: 0.6154 - val_recall: 0.6154 - val_auc: 0.7396
Epo

打印模型结构

In [15]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 112, 96, 3)] 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 114, 98, 3)   0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 56, 48, 64)   1728        zero_padding2d[0][0]             
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 56, 48, 64)   256         conv2d[0][0]                     
_______________________________________________________________________________________

评估模型效果

In [18]:
model.evaluate_generator(test_generator, verbose=1)

2/2 [==============================] - 0s 102ms/step - loss: 2.1719 - accuracy: 0.9667 - precision: 0.9667 - recall: 0.9667 - auc: 0.9667


[2.1719210147857666,
 0.9666666388511658,
 0.9666666388511658,
 0.9666666388511658,
 0.9666666984558105]

保存模型

In [17]:
model.save('./apert-normal-cnn.h5')